In [ ]:
# Import necessary python libraries
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg   # To embed Matplotlib figure into Tkinter GUI
import tkinter as tk                                              # To import Tkinter GUI elements
import numpy as np                                                # Numerical operations, arrays
import matplotlib.pyplot as plt                                   # To create plots and graphs
from scipy.integrate import solve_bvp                             # To solve ODEs (boundary value problem)
from scipy.integrate import odeint                                # To solve ODEs (initial value problem)
from matplotlib.colors import Normalize                           # To map numerical values to colours
from matplotlib.cm import ScalarMappable                          # To display a colour map
from matplotlib.patches import Ellipse                            # To draw ellipses


# Counter-current system    
def counter_current_solution (Th_in, Tc_in, mh, mc, Cph, Cpc, U, A_total, L):
    
    # To convert dT/dA to dT/dx
    dA_dx = A_total / L
    
    # Boundary conditions
    def bc(T0, TL):
        # T0 contains the hot and cold temperatures at x=0 so, T0 = [Th(0), Tc(0)]
        # TL contains the hot and cold temperatures at x=L so, TL = [Th(L), Tc(L)]
        # Hot inlet condition: Th(0) = Th_in and cold inlet condition: Tc(L) = Tc_in
        return [T0[0] - Th_in, TL[1] - Tc_in]  

    # ODE system
    def counter_odes(x, T):
        # T is the array that contains the variable T for the ODE solver.
        # It is unpacked to have two separate variables so we can access them separately in our ODEs
        Th, Tc = T                                  
        dTh_dx = -U * dA_dx * (Th - Tc) / (mh * Cph)      # Hot fluid
        dTc_dx = -U * dA_dx * (Th - Tc) / (mc * Cpc)      # Cold fluid (negative sign for counter-current)
        return [dTh_dx, dTc_dx]                           # Packages both derivatives as a coupled system
     
    x_span = np.linspace(0, L, 200)                       # 200 points between 0 and L
    guess = np.zeros((2, x_span.size))                    # Initialise guess array with 2 rows and size of x_span, full of zeros
    guess[0] = np.linspace(Th_in, Tc_in, x_span.size)     # Initial guess for hot stream, 200 linear points between Th_in and Tc_in
    guess [1] = guess [0]                                 # Same initial guess for cold stream

    # Solution 
    # Takes the differential equation system, the boundary conditions and solves the ODEs and evaluates the resultant function for x_span
    # Starts with the guess array and iterates until the residuals in bc are within the tolerance
    solution = solve_bvp(counter_odes, bc, x_span, guess, tol=1e-6)

    Th_profile = solution.y[0]                                  # Retrieves the hot stream profile
    Tc_profile = solution.y[1]                                  # Retrieves the cold stream profile 

    # Plotting
    fig_plot, ax = plt.subplots(figsize=(7.5,4.5))                  # Creates figure
    ax.plot(solution.x, Th_profile, 'r-', label="Hot stream →")     # Hot stream in red
    ax.plot(solution.x, Tc_profile, 'b-', label="Cold stream ←")    # Cold stream in blue

    # Labels and title 
    ax.set_xlabel("Length along heat exchanger (m)")              
    ax.set_ylabel("Temperature (°C)")
    ax.set_title("Counter-current Heat Exchanger: Temperature Profile")
    ax.grid(True)
    ax.legend()

    # Outlet values and heat transferred
    Th_out = Th_profile[-1]             # Last value in hot stream profile (hot outlet)
    Tc_out = Tc_profile[0]              # First value in cold stream profile (cold outlet)
    Q = mh * Cph * (Th_in - Th_out)     # Heat transferred 

    # Cylinders for pipes
    cmap = plt.cm.turbo                                             # Colour map
    norm = Normalize(vmin=min(Tc_profile), vmax=max(Th_profile))    # Normalises colour with numerical values of temperature

    fig_pipes, cyl_ax = plt.subplots(figsize=(5,5))                 # Creates figure for pipes
 
    # Hot stream cylinder (top)
    
    # Makes the 1D array 2D, with 1 row and 200 columns, then the row is replicated x30
    # So we can plot it as a thick recatngle with a constant colour vertically
    
    hot_img = np.tile(Th_profile[None,:], (30,1))  
    
    # imshow makes the 2D array an image
    # aspect=auto makes it stretch to fill the rectangle
    # cmap and norm apply the temperature colour map
    # extent defines the boundaries (two x values, two y values)
    # lower makes the bottom of the array align with the y values defined
    
    cyl_ax.imshow(hot_img, aspect="auto", cmap=cmap, norm=norm,
                  extent=[0.08,0.85,1.2,1.7], origin="lower")

    # Cold stream cylinder (bottom: placed lower in the axes)
    cold_img = np.tile(Tc_profile[None,:], (30,1))
    cyl_ax.imshow(cold_img, aspect="auto", cmap=cmap, norm=norm,
                  extent=[0.08,0.85,0.2,0.7], origin="lower")

    # Add cylinder ends with ellipses 
    for ycenter in [1.45, 0.45]:                                                 # y values of the centre of ellipses
        cyl_ax.add_patch(Ellipse((0.08, ycenter), width=0.05, height=0.5,
                                 facecolor="grey", edgecolor="black"))           # x values, width, height and colour
        cyl_ax.add_patch(Ellipse((0.85, ycenter), width=0.05, height=0.5,
                                 facecolor="grey", edgecolor="black"))

    # Set axis limits
    cyl_ax.set_xlim(-0.05,1.05)
    cyl_ax.set_ylim(-0.1,2.0)
    
    # Remove any numbers of ticks on the axes
    cyl_ax.set_xticks([])       
    cyl_ax.set_yticks([])  
    
    # Remove the axis frame
    for spine in cyl_ax.spines.values():
        spine.set_visible(False)
    
    # Labels at the inlets and outlets
    cyl_ax.text(0.03, 1.45, f"Hot inlet\n{Th_in:.1f}°C", va='center', ha='right', fontsize=10)
    cyl_ax.text(0.9, 1.45, f"Hot outlet\n{Th_out:.1f}°C", va='center', ha='left', fontsize=10)

    cyl_ax.text(0.9, 0.45, f"Cold inlet\n{Tc_in:.1f}°C", va='center', ha='left', fontsize=10)
    cyl_ax.text(0.03, 0.45, f"Cold outlet\n{Tc_out:.1f}°C", va='center', ha='right', fontsize=10)

    # Set the axis background colour (inside axes)
    cyl_ax.set_facecolor("lavender")

    # Set the figure background colour (around axes)
    fig_pipes.patch.set_facecolor("lavender")

    # Axis title
    cyl_ax.set_title("Pipes",  x=0.45, y=1.0, fontsize=14, fontweight="bold", fontfamily="Arial")

    # Colourbar

    # Create the colour mapper
    sm = ScalarMappable(cmap=cmap, norm=norm)   
    
    # Attach a horizontal colour bar to the pipe axes
    # fraction: how long the colour bar is compared to the axes
    # pad: distance between axes and colour bar
    cbar = fig_pipes.colorbar(sm, ax=cyl_ax, orientation='horizontal', fraction=0.08, pad=0.1)  

    # Add label to colour bar
    cbar.set_label("Temperature (°C)")
    
    # Display both figures
    return fig_plot, fig_pipes




# Co-current system
def co_current_solution(Th_in, Tc_in, mh, mc, Cph, Cpc, U, A_total, L):

    # To convert dT/dA to dT/dx
    dA_dx = A_total / L

    # ODE system
    def co_odes(T, x):
        # T is the array that contains the temperature variable for the ODE solver.
        # It is unpacked to have two separate variables so we can access them separately in our coupled ODEs
        Th, Tc = T
        dTh_dx = -U * dA_dx * (Th - Tc) / (mh * Cph)    # Hot fluid
        dTc_dx = U * dA_dx * (Th - Tc) / (mc * Cpc)     # Cold fluid
        return [dTh_dx, dTc_dx]                         # Packages both derivatives as a coupled system


    # Solution
    x_span = np.linspace(0, L, 200)           # Small divisions of x values along which the function will be integrated
    T0 = [Th_in, Tc_in]                       # Initial conditions
    solution = odeint(co_odes, T0, x_span)    # Integrates both functions and stores them in an array
    Th_profile = solution[:,0]                # Retrieves the hot stream profile
    Tc_profile = solution[:,1]                # Retrieves the cold stream profile


    fig_plot, ax = plt.subplots(figsize=(7.5,4.5))
    ax.plot(x_span, Th_profile, 'r-', label="Hot stream →")     # Hot stream in red
    ax.plot(x_span, Tc_profile, 'b-', label="Cold stream ←")    # Cold stream in blue

    # Labels and title 
    ax.set_xlabel("Length along heat exchanger (m)")              
    ax.set_ylabel("Temperature (°C)")
    ax.set_title("Co-current Heat Exchanger: Temperature Profile")
    ax.grid(True)
    ax.legend()

    # Outlet values and heat transferred
    Th_out = Th_profile[-1]             # Last value in hot stream profile (hot outlet)
    Tc_out = Tc_profile[-1]              # First value in cold stream profile (cold outlet)
    Q = mh * Cph * (Th_in - Th_out)     # Heat transferred 

    # Cylinders for pipes
    cmap = plt.cm.turbo                                             # Colour map
    norm = Normalize(vmin=min(Tc_profile), vmax=max(Th_profile))    # Normalises colour with numerical values of temperature

    fig_pipes, cyl_ax = plt.subplots(figsize=(5,5))                 # Creates figure for pipes
 
    # Hot stream cylinder (top)
    
    # Makes the 1D array 2D, with 1 row and 200 columns, then the row is replicated x30
    # So we can plot it as a thick recatngle with a constant colour vertically
    hot_img = np.tile(Th_profile[None,:], (30,1))  
    
    # imshow makes the 2D array an image
    # aspect=auto makes it stretch to fill the rectangle
    # cmap and norm apply the temperature colour map
    # extent defines the boundaries (two x values, two y values)
    # lower makes the bottom of the array align with the y values defined
    cyl_ax.imshow(hot_img, aspect="auto", cmap=cmap, norm=norm,
                  extent=[0.08,0.85,1.2,1.7], origin="lower")

    # Cold stream cylinder (bottom: placed lower in the axes)
    cold_img = np.tile(Tc_profile[None,:], (30,1))
    cyl_ax.imshow(cold_img, aspect="auto", cmap=cmap, norm=norm,
                  extent=[0.08,0.85,0.2,0.7], origin="lower")
   
        # Add cylinder ends with ellipses 
    for ycenter in [1.45, 0.45]:                                                 # y values of the centre of ellipses
        cyl_ax.add_patch(Ellipse((0.08, ycenter), width=0.05, height=0.5,
                                 facecolor="grey", edgecolor="black"))           # x values, width, height and colour
        cyl_ax.add_patch(Ellipse((0.85, ycenter), width=0.05, height=0.5,
                                 facecolor="grey", edgecolor="black"))

    # Set axis limits
    cyl_ax.set_xlim(-0.05,1.05)
    cyl_ax.set_ylim(-0.1,2.0)
    
    # Remove any numbers of ticks on the axes
    cyl_ax.set_xticks([])       
    cyl_ax.set_yticks([])  
    
    # Remove the axis frame
    for spine in cyl_ax.spines.values():
        spine.set_visible(False)

    # Labels at the inlets and outlets
    # Cold stream labels inverted for co-current
    cyl_ax.text(0.03, 1.45, f"Hot inlet\n{Th_in:.1f}°C", va='center', ha='right', fontsize=10)
    cyl_ax.text(0.9, 1.45, f"Hot outlet\n{Th_out:.1f}°C", va='center', ha='left', fontsize=10)

    cyl_ax.text(0.9, 0.45, f"Cold outlet\n{Tc_out:.1f}°C", va='center', ha='left', fontsize=10)
    cyl_ax.text(0.03, 0.45, f"Cold inlet\n{Tc_in:.1f}°C", va='center', ha='right', fontsize=10)

    # Set the axis background colour (inside axes)
    cyl_ax.set_facecolor("lavender")

    # Set the figure background colour (around axes)
    fig_pipes.patch.set_facecolor("lavender")

    # Axis title
    cyl_ax.set_title("Pipes", x=0.45, y=1.0, fontsize=14, fontweight="bold", fontfamily="Arial")

    # Colourbar

    # Create the colour mapper
    sm = ScalarMappable(cmap=cmap, norm=norm)   
    
    # Attach a horizontal colour bar to the pipe axes
    # fraction: how long the colour bar is compared to the axes
    # pad: distance between axes and colour bar
    cbar = fig_pipes.colorbar(sm, ax=cyl_ax, orientation='horizontal', fraction=0.08, pad=0.1)  

    # Add label to colour bar
    cbar.set_label("Temperature (°C)")
    
    # Display both figures
    return fig_plot, fig_pipes
   



def update_plot():
    try:
        Th_in = float(Th_in_entry.get())    # "get" reads the string in the entry box and "float" converts it to a floating point number
        Tc_in = float(Tc_in_entry.get())
        mh = float(mh_entry.get())
        mc = float(mc_entry.get())
        Cph = float(Cph_entry.get())
        Cpc = float(Cpc_entry.get())
        U = float(U_entry.get())
        A_total = float(A_total_entry.get())
        L = float(L_entry.get())
        
    except ValueError:
        return

    # If counter-current is chosen run the counter current solution
    if configuration.get() == "counter":
        fig_main, fig_pipes = counter_current_solution(Th_in, Tc_in, mh, mc, Cph, Cpc, U, A_total, L)

    # If co-current is chosen run the co current solution
    elif configuration.get() == "co":
        fig_main, fig_pipes = co_current_solution (Th_in, Tc_in, mh, mc, Cph, Cpc, U, A_total, L)

    # Place the two figures in the canvas (canvas is a widget that allows Matplotlib plots inside Tkinter GUI)
    canvas_main = FigureCanvasTkAgg(fig_main, master=root)
    canvas_main.get_tk_widget().grid(row=4, column=0, columnspan=5)

    canvas_pipes = FigureCanvasTkAgg(fig_pipes, master=root)
    canvas_pipes.get_tk_widget().grid(row=4, column=6, columnspan=4)  

    # Redraw to reflect any changes
    canvas_main.draw()
    canvas_pipes.draw()
    root.update()


# Create the GUI window
root = tk.Tk()                          
root.title("Heat Exchanger Simulator")
root.config(bg='lavender')

# Entry boxes and buttons

# Title for entry boxes

# Variables that can be changed
tk.Label(root, text="    Variables that can be changed   \n ", font= ("Arial",14, "bold"), bg='lavender').grid(row=0, column=1, columnspan=3)

# Co-current vs Counter-current
tk.Label(root, text="    Configuration   \n ", font= ("Arial",14, "bold"), bg='lavender').grid(row=0, column=7, columnspan=2)

# Hot stream inlet temperature 
tk.Label(root, text="Hot stream inlet \ntemperature [°C]", bg='lavender', font=("Calibri", 12)).grid(row=1, column=0, pady=10)    # Label and its location
Th_in_entry = tk.Entry(root, width=8, bd=0.5, relief="solid")                                                                     # Creates entry box with specified width
Th_in_entry.insert(0, "400")                                                                                                      # Inserts a default value
Th_in_entry.grid(row=1, column=1)                                                                                                 # Location of entry box

# Cold stream inlet temperature
tk.Label(root, text= "Cold stream inlet \ntemperature [°C]", bg='lavender', font=("Calibri", 12)).grid(row=1, column=2)
Tc_in_entry = tk.Entry(root, width=8, bd=0.5, relief="solid")
Tc_in_entry.insert(0, "30")
Tc_in_entry.grid(row=1, column=3)

# Mass flow rate of hot stream
tk.Label(root, text="Mass flow rate of \nhot stream [kg/s]", bg='lavender', font=("Calibri", 12)).grid(row=1, column=4)
mh_entry = tk.Entry(root, width=8, bd=0.5, relief="solid")
mh_entry.insert(0, "1.0")
mh_entry.grid(row=1, column=5)

# Mass flow rate of cold stream
tk.Label(root, text="Mass flow rate of \ncold stream [kg/s]", bg='lavender', font=("Calibri", 12)).grid(row=2, column=0, pady=10)
mc_entry = tk.Entry(root, width=8, bd=0.5, relief="solid")
mc_entry.insert(0, "1.2")
mc_entry.grid(row=2, column=1)

# Heat capacity of hot stream
tk.Label(root, text="Heat capacity of \nhot stream [J/kg°C]", bg='lavender', font=("Calibri", 12)).grid(row=2, column=2)
Cph_entry = tk.Entry(root, width=8, bd=0.5, relief="solid")
Cph_entry.insert(0, "4180")
Cph_entry.grid(row=2, column=3)

# Heat capacity of cold stream
tk.Label(root, text="Heat capacity of \ncold stream [J/kg°C]", bg='lavender', font=("Calibri", 12)).grid(row=2, column=4)
Cpc_entry = tk.Entry(root, width=8, bd=0.5, relief="solid")
Cpc_entry.insert(0, "4180")
Cpc_entry.grid(row=2, column=5)

# Overall heat transfer coefficient
tk.Label(root, text="Overall heat transfer \ncoefficient [W/m²°C]", bg='lavender', font=("Calibri", 12)).grid(row=3, column=0, pady=10)
U_entry = tk.Entry(root, width=8, bd=0.5, relief="solid")
U_entry.insert(0, "1000")
U_entry.grid(row=3, column=1)

# Total heat exchanger area
tk.Label(root, text="Total heat exchanger\n area [m²]", bg='lavender', font=("Calibri", 12)).grid(row=3, column=2)
A_total_entry = tk.Entry(root, width=8, bd=0.5, relief="solid")
A_total_entry.insert(0, "20")
A_total_entry.grid(row=3, column=3)

# Length of heat exchanger
tk.Label(root, text="Length of heat \nexchanger [m]", bg='lavender', font=("Calibri", 12)).grid(row=3, column=4)
L_entry = tk.Entry(root, width=8, bd=0.5, relief="solid")
L_entry.insert(0, "10")
L_entry.grid(row=3, column=5)

# Radio buttons for configuration
configuration = tk.StringVar(value="counter")

# Co-current
rb_co = tk.Radiobutton(root, text="Co-Current", variable=configuration, value="co", font=("Calibri", 12), bg='lavender').grid(row=1, column=7,columnspan=2)

# Counter-current 
rb_counter = tk.Radiobutton(root, text="Counter-Current", variable=configuration, value="counter", font=("Calibri", 12), bg='lavender').grid(row=2, column=7, columnspan=2)

# Initial display
fig_main, fig_pipes = counter_current_solution(400, 30, 1.0, 1.2, 4180, 4180, 1000, 20, 10)

canvas_main = FigureCanvasTkAgg(fig_main, master=root)
canvas_main.get_tk_widget().grid(row=4, column=0, columnspan=5)

canvas_pipes = FigureCanvasTkAgg(fig_pipes, master=root)
canvas_pipes.get_tk_widget().grid(row=4, column=6, columnspan=4)   


# Update button
tk.Button(root, text="Update",  font=("Calibri", 13),command=update_plot).grid(row=3, column=7, columnspan=2, rowspan=1)

# Event loop that keeps the GUI responsive and running until the window is closed
root.mainloop()  
